# 学習済みの畳み込みベースを用いたCNNの例題
# Dog or Cat判別
認識率を上げるため、既に学習されているモデルを利用する方法がある。

CNNの場合は、畳み込み層の内容を他の学習モデル同等にし、全結合層のみ1から学習させることが多い。

理由としては畳み込み層は汎用的で判別する対象が異なる場合でも利用できる場合が多いため。

全結合層はそれぞれの判別器に特化しているため再利用できる機会は少ない。

In [1]:
%matplotlib inline

import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.applications import VGG16

np.random.seed(20210101)
tf.random.set_seed(20210101)

print("tensorflow version : " + tf.__version__)

tensorflow version : 2.3.1


# Get the trained model VGG16

In [2]:
conv_base = VGG16(weights='imagenet',
                  #畳み込みベースの層のみ利用し、全結合層は利用しない
                  include_top=False,
                  input_shape=(150, 150, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

# Create the learning model using VGG16 conv layer

In [3]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


# Freeze the conv model parameters

In [4]:
#訓練を行う前にconv modelの凍結を行い、VGG16用に学習されたパラメータから変更しないようにする
#凍結しない場合、下の学習処理でconv部分のパラメータも変更されてしまうため
conv_base.trainable = False

# Modify the input data

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

base_dir = '.\\datasets\\cnn_dog_vs_cat'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

#学習データが少ないため、回転・平行移動・拡大等でデータ拡張(水増し)を行う
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

#検証データは水増しを行うべきではない
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Training

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

training = model.fit_generator(
      train_generator,
      #steps_per_epoch=100,
      steps_per_epoch=10,
      #epochs=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=50)

#正答率
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#loss
plt.plot(training.history['loss'])
plt.plot(training.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Save the model

In [ ]:
model.save('models\\cnn_dog_vs_cat_using_vgg16_conv.h5')
model.save('models/cnn_dog_vs_cat_using_vgg16_conv.h5')

print(model.input)
print(model.output)

# Fine turning
畳み込み層の後段のいくつかを解凍し、新たに追加した部分(全結合層)と共に学習を行う。

畳み込み層の入力側の層はより汎用的で再利用可能な特徴量を処理しており、
後段の層はより非汎用的であるため。

今回はblock5の内容をFine turninする。

Fine turningを行う際には下記の手順で行う。

1.訓練済みのモデルの最後に独自のモデルを追加する

2.訓練済みのモデルを凍結する

3.訓練を行う

4.訓練済みのモデルの一部の層を解凍する

5.解凍した層と独自のモデルの学習を再度行う

In [6]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
# block5のみ解凍を行う
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
#Fine turning時はRMSpropと低い学習率を用いることが多い
#理由としては学習による変化量を制限したいため、変化量が大きいとパラメータが破壊されてしまうから
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])

training = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

#正答率
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#loss
plt.plot(training.history['loss'])
plt.plot(training.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('models\\cnn_dog_vs_cat_using_vgg16_conv2.h5')
model.save('models/cnn_dog_vs_cat_using_vgg16_conv2.h5')

print(model.input)
print(model.output)

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)